In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

import os
import openai
import sys
import datetime
sys.path.append('../..')


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']


current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

persist_directory = 'docs/chroma/'

def load_db(file, chain_type, k):
    print(file)
    loaders = [
        # Duplicate documents on purpose - messy data
#         PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture02.pdf"),
        PyPDFLoader(file)
    ]
    docList = []
    for loader in loaders:
        print('count')
        docList.extend(loader.load())

    print(len(docList))
        
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1500,
        chunk_overlap = 150
    )    
    docs = text_splitter.split_documents(docList)
    
    # define embedding
    embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)
    
     # create vector database from data
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embedding,
        persist_directory=persist_directory
    )
    retriever=vectordb.as_retriever()
    
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    # define retriever
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=openai.api_key), 
        chain_type=chain_type, 
        retriever=retriever, 
        memory=memory
    )
    return qa 

import param

class cbfs(param.Parameterized):
#     chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.loaded_file = "sw.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    

    def convchain(self, query):
        result = self.qa({"question": query})
        self.answer = result['answer'] 
        return self.answer
    
    def clr_history(self,count=0):
        self.chat_history = []
        return 

cb = cbfs()
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/api', methods=['GET'])
def get_string():
    try:
        # Get the input string from the query parameters
        input_string = request.args.get('query')

        # Process the input string (you can add your logic here)
        output_string = process_input(input_string)

        # Return the output as JSON
        response = {'output_string': output_string}
        return jsonify(response)
    except Exception as e:
        return jsonify({'error': str(e)}), 400

def process_input(input_string):
    return cb.convchain(input_string)
    
if __name__ == '__main__':
    app.run()

gpt-3.5-turbo
sw.pdf
count
33
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Sep/2023 10:04:25] "GET /api?query='what%20is%20SW' HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2023 10:04:54] "GET /api?query=shareworks HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2023 10:04:59] "GET /api?query=shareworks HTTP/1.1" 200 -
